# 궤적 일관성 증류-LoRA

궤적 일관성 증류(TCD)를 사용하면 모델이 더 적은 단계로 더 높은 품질과 더 자세한 이미지를 생성할 수 있습니다. 또한 증류 과정에서 효과적인 오류 완화 덕분에 TCD는 많은 추론 단계 조건에서도 우수한 성능을 보여줍니다.

TCD의 주요 장점은 다음과 같습니다.

- 교사보다 우수: TCD는 작고 큰 추론 단계 모두에서 우수한 생성 품질을 보여주며 Stable Diffusion XL(SDXL)을 사용한 [DPM-Solver++(2S)](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../../api/schedulers/multistep_dpm_solver)의 성능을 능가합니다. TCD 학습 중에는 추가 판별기 또는 LPIPS 감독이 포함되지 않습니다.

- 유연한 추론 단계: TCD 샘플링의 추론 단계는 이미지 품질에 부정적인 영향을 미치지 않고 자유롭게 조정할 수 있습니다.

- 세부 수준 자유롭게 변경: 추론 중에 단일 하이퍼파라미터인 *감마*를 사용하여 이미지의 세부 수준을 조정할 수 있습니다.

> [!TIP]
> TCD에 대한 자세한 기술 내용은 [논문](https://huggingface.co/papers/2402.19159) 또는 공식 [프로젝트 페이지](https://mhh0318.github.io/tcd/)를 참조하십시오.

SDXL과 같은 대규모 모델의 경우 TCD는 [LoRA](https://huggingface.co/docs/peft/conceptual_guides/adapter#low-rank-adaptation-lora)를 사용하여 메모리 사용량을 줄여 학습합니다. 이는 동일한 기본 모델을 공유하는 한 추가 학습 없이 서로 다른 미세 조정 모델 간에 LoRA를 재사용할 수 있기 때문에 유용합니다.



이 가이드에서는 텍스트-이미지 및 인페인팅과 같은 다양한 작업에 대해 TCD-LoRA를 사용하여 추론을 수행하는 방법과 TCD-LoRA를 다른 어댑터와 쉽게 결합하는 방법을 보여줍니다. 시작하려면 아래 표에서 지원되는 기본 모델과 해당 TCD-LoRA 체크포인트 중 하나를 선택하십시오.

| 기본 모델                                                                                      | TCD-LoRA 체크포인트                                            |
|-------------------------------------------------------------------------------------------------|----------------------------------------------------------------|
| [stable-diffusion-v1-5](https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5)                  | [TCD-SD15](https://huggingface.co/h1t/TCD-SD15-LoRA)           |
| [stable-diffusion-2-1-base](https://huggingface.co/stabilityai/stable-diffusion-2-1-base)       | [TCD-SD21-base](https://huggingface.co/h1t/TCD-SD21-base-LoRA) |
| [stable-diffusion-xl-base-1.0](https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0) | [TCD-SDXL](https://huggingface.co/h1t/TCD-SDXL-LoRA)           |


더 나은 LoRA 지원을 위해 [PEFT](https://github.com/huggingface/peft)가 설치되어 있는지 확인하십시오.

```bash
pip install -U peft
```

## 일반 작업

이 가이드에서는 [StableDiffusionXLPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/stable_diffusion_xl#diffusers.StableDiffusionXLPipeline)과 [TCDScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/tcd#diffusers.TCDScheduler)를 사용합니다. [load_lora_weights()](https://huggingface.co/docs/diffusers/main/en/api/loaders/lora#diffusers.loaders.StableDiffusionLoraLoaderMixin.load_lora_weights) 메서드를 사용하여 SDXL 호환 TCD-LoRA 가중치를 로드합니다.

TCD-LoRA 추론 시 염두에 두어야 할 몇 가지 팁은 다음과 같습니다.

- `num_inference_steps`를 4에서 50 사이로 유지합니다.
- `eta`(각 단계에서 확률성을 제어하는 데 사용됨)를 0에서 1 사이로 설정합니다. 추론 단계 수를 늘릴 때 더 높은 `eta`를 사용해야 하지만, [TCDScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/tcd#diffusers.TCDScheduler)에서 `eta`가 클수록 이미지가 흐릿해지는 단점이 있습니다. 좋은 결과를 얻으려면 0.3 값을 권장합니다.

<hfoptions id="tasks">
<hfoption id="text-to-image">

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline, TCDScheduler

device = "cuda"
base_model_id = "stabilityai/stable-diffusion-xl-base-1.0"
tcd_lora_id = "h1t/TCD-SDXL-LoRA"

pipe = StableDiffusionXLPipeline.from_pretrained(base_model_id, torch_dtype=torch.float16, variant="fp16").to(device)
pipe.scheduler = TCDScheduler.from_config(pipe.scheduler.config)

pipe.load_lora_weights(tcd_lora_id)
pipe.fuse_lora()

prompt = "Painting of the orange cat Otto von Garfield, Count of Bismarck-Schönhausen, Duke of Lauenburg, Minister-President of Prussia. Depicted wearing a Prussian Pickelhaube and eating his favorite meal - lasagna."

image = pipe(
    prompt=prompt,
    num_inference_steps=4,
    guidance_scale=0,
    eta=0.3,
    generator=torch.Generator(device=device).manual_seed(0),
).images[0]

![](https://github.com/jabir-zheng/TCD/raw/main/assets/demo_image.png)

</hfoption>

<hfoption id="inpainting">

In [ ]:
import torch
from diffusers import AutoPipelineForInpainting, TCDScheduler
from diffusers.utils import load_image, make_image_grid

device = "cuda"
base_model_id = "diffusers/stable-diffusion-xl-1.0-inpainting-0.1"
tcd_lora_id = "h1t/TCD-SDXL-LoRA"

pipe = AutoPipelineForInpainting.from_pretrained(base_model_id, torch_dtype=torch.float16, variant="fp16").to(device)
pipe.scheduler = TCDScheduler.from_config(pipe.scheduler.config)

pipe.load_lora_weights(tcd_lora_id)
pipe.fuse_lora()

img_url = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo.png"
mask_url = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo_mask.png"

init_image = load_image(img_url).resize((1024, 1024))
mask_image = load_image(mask_url).resize((1024, 1024))

prompt = "a tiger sitting on a park bench"

image = pipe(
  prompt=prompt,
  image=init_image,
  mask_image=mask_image,
  num_inference_steps=8,
  guidance_scale=0,
  eta=0.3,
  strength=0.99,  # `strength`는 1.0 미만으로 사용해야 합니다.
  generator=torch.Generator(device=device).manual_seed(0),
).images[0]

grid_image = make_image_grid([init_image, mask_image, image], rows=1, cols=3)

![](https://github.com/jabir-zheng/TCD/raw/main/assets/inpainting_tcd.png)


</hfoption>
</hfoptions>

## 커뮤니티 모델

TCD-LoRA는 또한 많은 커뮤니티에서 미세 조정한 모델 및 플러그인과 함께 작동합니다. 예를 들어, 애니메이션 이미지를 생성하기 위해 커뮤니티에서 미세 조정한 SDXL 버전인 [animagine-xl-3.0](https://huggingface.co/cagliostrolab/animagine-xl-3.0) 체크포인트를 로드합니다.

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline, TCDScheduler

device = "cuda"
base_model_id = "cagliostrolab/animagine-xl-3.0"
tcd_lora_id = "h1t/TCD-SDXL-LoRA"

pipe = StableDiffusionXLPipeline.from_pretrained(base_model_id, torch_dtype=torch.float16, variant="fp16").to(device)
pipe.scheduler = TCDScheduler.from_config(pipe.scheduler.config)

pipe.load_lora_weights(tcd_lora_id)
pipe.fuse_lora()

prompt = "A man, clad in a meticulously tailored military uniform, stands with unwavering resolve. The uniform boasts intricate details, and his eyes gleam with determination. Strands of vibrant, windswept hair peek out from beneath the brim of his cap."

image = pipe(
    prompt=prompt,
    num_inference_steps=8,
    guidance_scale=0,
    eta=0.3,
    generator=torch.Generator(device=device).manual_seed(0),
).images[0]

![](https://github.com/jabir-zheng/TCD/raw/main/assets/animagine_xl.png)

TCD-LoRA는 다른 스타일로 학습된 다른 LoRA도 지원합니다. 예를 들어, [TheLastBen/Papercut_SDXL](https://huggingface.co/TheLastBen/Papercut_SDXL) LoRA를 로드하고 `~loaders.UNet2DConditionLoadersMixin.set_adapters` 메서드를 사용하여 TCD-LoRA와 융합합니다.

> [!TIP]
> 효율적인 병합 방법에 대한 자세한 내용은 [LoRA 병합](https://huggingface.co/docs/diffusers/main/en/using-diffusers/merge_loras) 가이드를 참조하십시오.

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline
from scheduling_tcd import TCDScheduler

device = "cuda"
base_model_id = "stabilityai/stable-diffusion-xl-base-1.0"
tcd_lora_id = "h1t/TCD-SDXL-LoRA"
styled_lora_id = "TheLastBen/Papercut_SDXL"

pipe = StableDiffusionXLPipeline.from_pretrained(base_model_id, torch_dtype=torch.float16, variant="fp16").to(device)
pipe.scheduler = TCDScheduler.from_config(pipe.scheduler.config)

pipe.load_lora_weights(tcd_lora_id, adapter_name="tcd")
pipe.load_lora_weights(styled_lora_id, adapter_name="style")
pipe.set_adapters(["tcd", "style"], adapter_weights=[1.0, 1.0])

prompt = "papercut of a winter mountain, snow"

image = pipe(
    prompt=prompt,
    num_inference_steps=4,
    guidance_scale=0,
    eta=0.3,
    generator=torch.Generator(device=device).manual_seed(0),
).images[0]

![](https://github.com/jabir-zheng/TCD/raw/main/assets/styled_lora.png)

## 어댑터

TCD-LoRA는 매우 다재다능하며 ControlNet, IP-Adapter 및 AnimateDiff와 같은 다른 어댑터 유형과 결합할 수 있습니다.

<hfoptions id="adapters">
<hfoption id="ControlNet">

### 깊이 제어망

In [ ]:
import torch
import numpy as np
from PIL import Image
from transformers import DPTImageProcessor, DPTForDepthEstimation
from diffusers import ControlNetModel, StableDiffusionXLControlNetPipeline
from diffusers.utils import load_image, make_image_grid
from scheduling_tcd import TCDScheduler

device = "cuda"
depth_estimator = DPTForDepthEstimation.from_pretrained("Intel/dpt-hybrid-midas").to(device)
feature_extractor = DPTImageProcessor.from_pretrained("Intel/dpt-hybrid-midas")

def get_depth_map(image):
    image = feature_extractor(images=image, return_tensors="pt").pixel_values.to(device)
    with torch.no_grad(), torch.autocast(device):
        depth_map = depth_estimator(image).predicted_depth

    depth_map = torch.nn.functional.interpolate(
        depth_map.unsqueeze(1),
        size=(1024, 1024),
        mode="bicubic",
        align_corners=False,
    )
    depth_min = torch.amin(depth_map, dim=[1, 2, 3], keepdim=True)
    depth_max = torch.amax(depth_map, dim=[1, 2, 3], keepdim=True)
    depth_map = (depth_map - depth_min) / (depth_max - depth_min)
    image = torch.cat([depth_map] * 3, dim=1)

    image = image.permute(0, 2, 3, 1).cpu().numpy()[0]
    image = Image.fromarray((image * 255.0).clip(0, 255).astype(np.uint8))
    return image

base_model_id = "stabilityai/stable-diffusion-xl-base-1.0"
controlnet_id = "diffusers/controlnet-depth-sdxl-1.0"
tcd_lora_id = "h1t/TCD-SDXL-LoRA"

controlnet = ControlNetModel.from_pretrained(
    controlnet_id,
    torch_dtype=torch.float16,
    variant="fp16",
)
pipe = StableDiffusionXLControlNetPipeline.from_pretrained(
    base_model_id,
    controlnet=controlnet,
    torch_dtype=torch.float16,
    variant="fp16",
)
pipe.enable_model_cpu_offload()

pipe.scheduler = TCDScheduler.from_config(pipe.scheduler.config)

pipe.load_lora_weights(tcd_lora_id)
pipe.fuse_lora()

prompt = "stormtrooper lecture, photorealistic"

image = load_image("https://huggingface.co/lllyasviel/sd-controlnet-depth/resolve/main/images/stormtrooper.png")
depth_image = get_depth_map(image)

controlnet_conditioning_scale = 0.5  # 좋은 일반화를 위해 권장됨

image = pipe(
    prompt,
    image=depth_image,
    num_inference_steps=4,
    guidance_scale=0,
    eta=0.3,
    controlnet_conditioning_scale=controlnet_conditioning_scale,
    generator=torch.Generator(device=device).manual_seed(0),
).images[0]

grid_image = make_image_grid([depth_image, image], rows=1, cols=2)

![](https://github.com/jabir-zheng/TCD/raw/main/assets/controlnet_depth_tcd.png)

### 캐니 컨트롤넷

In [ ]:
import torch
from diffusers import ControlNetModel, StableDiffusionXLControlNetPipeline
from diffusers.utils import load_image, make_image_grid
from scheduling_tcd import TCDScheduler

device = "cuda"
base_model_id = "stabilityai/stable-diffusion-xl-base-1.0"
controlnet_id = "diffusers/controlnet-canny-sdxl-1.0"
tcd_lora_id = "h1t/TCD-SDXL-LoRA"

controlnet = ControlNetModel.from_pretrained(
    controlnet_id,
    torch_dtype=torch.float16,
    variant="fp16",
)
pipe = StableDiffusionXLControlNetPipeline.from_pretrained(
    base_model_id,
    controlnet=controlnet,
    torch_dtype=torch.float16,
    variant="fp16",
)
pipe.enable_model_cpu_offload()

pipe.scheduler = TCDScheduler.from_config(pipe.scheduler.config)

pipe.load_lora_weights(tcd_lora_id)
pipe.fuse_lora()

prompt = "ultrarealistic shot of a furry blue bird"

canny_image = load_image("https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main/sd_controlnet/bird_canny.png")

controlnet_conditioning_scale = 0.5  # 좋은 일반화를 위해 권장됨

image = pipe(
    prompt,
    image=canny_image,
    num_inference_steps=4,
    guidance_scale=0,
    eta=0.3,
    controlnet_conditioning_scale=controlnet_conditioning_scale,
    generator=torch.Generator(device=device).manual_seed(0),
).images[0]

grid_image = make_image_grid([canny_image, image], rows=1, cols=2)

![](https://github.com/jabir-zheng/TCD/raw/main/assets/controlnet_canny_tcd.png)

<Tip>
이 예제의 추론 매개변수는 모든 예제에 적용되지 않을 수 있으므로 `num_inference_steps`, `guidance_scale`, `controlnet_conditioning_scale` 및 `cross_attention_kwargs` 매개변수에 대해 다른 값을 시도하고 가장 적합한 값을 선택하는 것이 좋습니다.
</Tip>

</hfoption>
<hfoption id="IP-Adapter">

이 예제에서는 [IP-Adapter](https://github.com/tencent-ailab/IP-Adapter/tree/main) 및 SDXL과 함께 TCD-LoRA를 사용하는 방법을 보여줍니다.

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline
from diffusers.utils import load_image, make_image_grid

from ip_adapter import IPAdapterXL
from scheduling_tcd import TCDScheduler

device = "cuda"
base_model_path = "stabilityai/stable-diffusion-xl-base-1.0"
image_encoder_path = "sdxl_models/image_encoder"
ip_ckpt = "sdxl_models/ip-adapter_sdxl.bin"
tcd_lora_id = "h1t/TCD-SDXL-LoRA"

pipe = StableDiffusionXLPipeline.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,
    variant="fp16"
)
pipe.scheduler = TCDScheduler.from_config(pipe.scheduler.config)

pipe.load_lora_weights(tcd_lora_id)
pipe.fuse_lora()

ip_model = IPAdapterXL(pipe, image_encoder_path, ip_ckpt, device)

ref_image = load_image("https://raw.githubusercontent.com/tencent-ailab/IP-Adapter/main/assets/images/woman.png").resize((512, 512))

prompt = "best quality, high quality, wearing sunglasses"

image = ip_model.generate(
    pil_image=ref_image,
    prompt=prompt,
    scale=0.5,
    num_samples=1,
    num_inference_steps=4,
    guidance_scale=0,
    eta=0.3,
    seed=0,
)[0]

grid_image = make_image_grid([ref_image, image], rows=1, cols=2)

![](https://github.com/jabir-zheng/TCD/raw/main/assets/ip_adapter.png)



</hfoption>
<hfoption id="AnimateDiff">

`AnimateDiff`를 사용하면 스테이블 디퓨전 모델을 사용하여 이미지를 애니메이션으로 만들 수 있습니다. TCD-LoRA는 이미지 품질을 저하시키지 않고 프로세스를 크게 가속화할 수 있습니다. TCD-LoRA와 AnimateDiff를 사용한 애니메이션 품질은 더욱 선명한 결과를 보여줍니다.

In [ ]:
import torch
from diffusers import MotionAdapter, AnimateDiffPipeline, DDIMScheduler
from scheduling_tcd import TCDScheduler
from diffusers.utils import export_to_gif

adapter = MotionAdapter.from_pretrained("guoyww/animatediff-motion-adapter-v1-5")
pipe = AnimateDiffPipeline.from_pretrained(
    "frankjoshua/toonyou_beta6",
    motion_adapter=adapter,
).to("cuda")

# TCDScheduler 설정
pipe.scheduler = TCDScheduler.from_config(pipe.scheduler.config)

# TCD LoRA 로드
pipe.load_lora_weights("h1t/TCD-SD15-LoRA", adapter_name="tcd")
pipe.load_lora_weights("guoyww/animatediff-motion-lora-zoom-in", weight_name="diffusion_pytorch_model.safetensors", adapter_name="motion-lora")

pipe.set_adapters(["tcd", "motion-lora"], adapter_weights=[1.0, 1.2])

prompt = "best quality, masterpiece, 1girl, looking at viewer, blurry background, upper body, contemporary, dress"
generator = torch.manual_seed(0)
frames = pipe(
    prompt=prompt,
    num_inference_steps=5,
    guidance_scale=0,
    cross_attention_kwargs={"scale": 1},
    num_frames=24,
    eta=0.3,
    generator=generator
).frames[0]
export_to_gif(frames, "animation.gif")

![](https://github.com/jabir-zheng/TCD/raw/main/assets/animation_example.gif)

</hfoption>
</hfoptions>